In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, RadioMaterial, LambertianPattern
from sionna.constants import PI

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [ ]:
# Load integrated scene
scene = load_scene("../../scenes/trainstation_deutscheban/trainstation_deutscheban.xml")

In [ ]:
da = 0.5
M = 8
N = 4

scene._clear()

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=N,
                             num_cols=M,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="tr38901",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="tr38901",
                             polarization="V")
                             #polarization="cross")

# Create transmitter
tx_pos = [95.8,114.6,18.11]
tx = Transmitter(name="tx",
                 position=tx_pos)

# Add transmitter instance to scene
scene.add(tx)
tx_pos = [94,60,13.41]

# Receiver positions
rx_pos = [43.9394, 125.376, 5.16987]
rx = Receiver(name="rx",
              position=rx_pos)

scene.add(rx)
# Receiver positions

sampling_frequency = 10e6
sample_m = 62.0 / sampling_frequency

c0 = sionna.SPEED_OF_LIGHT

In [ ]:
tx.look_at(rx) # Transmitter points towards receiver
rx.look_at(tx)

In [ ]:
fc = 3.6e9
scene.frequency = fc # in Hz; implicitly updates RadioMaterials

scene.synthetic_array = False # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [ ]:
scene._scene_params

In [ ]:
scene.radio_materials

In [ ]:
# Compute propagation paths
paths = scene.compute_paths(max_depth=10,
                            num_samples=1e6)  # Number of rays shot into directions defined
                                              # by a Fibonacci spheree , too few rays can
                                              # lead to missing paths

paths.normalize_delays = False

# FR2 setup
subcarrier_spacing = 120e3 # numerology 3
fft_size = 64 # for testing

paths.apply_doppler(sampling_frequency=subcarrier_spacing, # Set to subcarrier spacing
                    num_time_steps=10, # Number of time steps
                    tx_velocities=[0,0,0], # We can set additional tx speeds
                    rx_velocities=[0,0,0]) # Or rx speeds

a, tau = paths.cir()

In [ ]:
scene.preview(paths, show_devices=True, show_paths=True) # Use the mouse to focus on the visualized paths

In [ ]:
# Compute frequencies of subcarriers and center around carrier frequency
frequencies = subcarrier_frequencies(fft_size, subcarrier_spacing)

# Compute the frequency response of the channel at frequencies.
h_freq = cir_to_ofdm_channel(frequencies,
                             a,
                             tau,
                             normalize=False) # Non-normalized includes path-loss

In [ ]:
# (:,:,:,:,antenna,time_step,freq_bin)
print(h_freq.shape)

In [ ]:
# apply DFT beamforming (in frequency domain)
m = np.arange(0, M)
n = np.arange(0, N)

angle_azi = 30
angle_ele = 10

theta = PI/180.0 * angle_azi
phi = PI/180.0 * angle_ele
#array_response = tf.cast(tf.exp(-2j * np.pi * fc / c0 * m * np.sin(theta)), dtype=tf.complex64) # tensorflow test
array_response_azi = np.exp(-2j * PI * fc / c0 * da *  m * np.sin(theta))
array_response_ele = np.exp(-2j * PI * fc / c0 * da *  n * np.sin(phi))

array_response_2d = np.kron(array_response_azi, np.transpose(array_response_ele))

print(array_response_2d)

h_freq_tilde = np.zeros([1,1,1,1,1,10,64], dtype=np.complex64)
                        
                        
for i in range(0,10,1): 
    h_freq_tilde[:,:,:,:,:,i,:] = np.matmul(array_response_2d, np.squeeze(h_freq[:,:,:,:,:,i,:] ))

print(h_freq_tilde.shape)

In [ ]:
# print beamforming pattern
theta_range = np.linspace(-PI/2, PI/2, 100)
phi_range = np.linspace(-PI/2, PI/2, 100)

#theta_range = theta_range[..., None]
#phi_range = phi_range[..., None]


wavelength = c0/fc

AF_azi = np.sin(M * PI * da * (np.sin(theta_range) - np.sin(theta) ) ) / (M * np.sin( PI * da * (np.sin(theta_range) - np.sin(theta) ) ) )
AF_ele = np.sin(N * PI * da * (np.sin(phi_range) - np.sin(phi) ) ) / (N * np.sin( PI * da * (np.sin(phi_range) - np.sin(phi) ) ) ) 

# print(AF_azi.shape)
# print(AF_ele.shape)

AF_2D = np.kron(AF_azi, np.atleast_2d(AF_ele).T)

print(AF_2D.shape)

# AF_dB = 10*np.log10(np.abs(AF))
plt.figure()
plt.plot(theta_range / PI * 180, AF_ele, "-")
plt.xlabel("Angle")
plt.ylabel("Array response")

theta_range, phi_range = np.meshgrid(theta_range, phi_range)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(theta_range / PI * 180, phi_range  / PI * 180, AF_2D)
# ax.set_aspect('equal')


In [ ]:
# compare frequency responses

# Visualize results
plt.figure()
plt.plot(np.abs(h_freq)[0,0,0,0,0,0,:], "-")
plt.plot(np.abs(h_freq_tilde)[0,0,0,0,0,0,:], "--")
plt.xlabel("Subcarrier index")
plt.ylabel("Channel frequency response")


In [ ]:
test_antenna = sionna.rt.Antenna("tr38901", "V")

In [ ]:
def bf_pattern(theta, phi, slant_angle=0.0,
                    polarization_model=2, dtype=tf.complex64):
    r"""
    Antenna pattern from 3GPP TR 38.901 (Table 7.3-1) [TR38901]_

    Input
    -----
    theta: array_like, float
        Zenith angles wrapped within [0,pi] [rad]

    phi: array_like, float
        Azimuth angles wrapped within [-pi, pi) [rad]

    slant_angle: float
        Slant angle of the linear polarization [rad].
        A slant angle of zero means vertical polarization.

    polarization_model: int, one of [1,2]
        Polarization model to be used. Options `1` and `2`
        refer to :func:`~sionna.rt.antenna.polarization_model_1`
        and :func:`~sionna.rt.antenna.polarization_model_2`,
        respectively.
        Defaults to `2`.

    dtype : tf.complex64 or tf.complex128
        Datatype.
        Defaults to `tf.complex64`.

    Output
    ------
    c_theta: array_like, complex
        Zenith pattern

    c_phi: array_like, complex
        Azimuth pattern

    """
    rdtype = dtype.real_dtype
    theta = tf.cast(theta, rdtype)
    phi = tf.cast(phi, rdtype)

    slant_angle = tf.cast(slant_angle, rdtype)
    M = 8
    N = 8
    da = 0.5
    theta_p = -PI/6#0.0
    phi_p = 0.0#-PI/3

    #wrap theta_p to [-PI/2, PI/2]
    theta = theta+PI/2

    # Wrap phi to [-PI,PI]
    phi = tf.math.floormod(phi+PI, 2*PI)-PI

    if not theta.shape==phi.shape:
        raise ValueError("theta and phi must have the same shape.")
    if polarization_model not in [1,2]:
        raise ValueError("polarization_model must be 1 or 2")
    a_azi = tf.zeros_like(phi)

    phi = tf.where(phi<-PI/2, 0.0, phi)
    phi = tf.where(phi>PI/2, 0.0, phi)

    a_azi = tf.math.divide_no_nan(tf.sin( M * (PI * da * tf.sin(phi) - PI*da * tf.sin(phi_p) ) ) , (M * tf.sin( PI * da * tf.sin(phi) - PI * da * tf.sin(phi_p) )))
    a_azi = tf.where(phi<-PI/2, 0.0, a_azi)
    a_azi = tf.where(phi>PI/2, 0.0, a_azi)
    
    a_ele = tf.math.divide_no_nan(tf.sin( N * (PI * da * tf.sin(theta) - PI*da * tf.sin(theta_p) ) ), (N * tf.sin( PI * da * tf.sin(theta) - PI * da * tf.sin(theta_p) )))

    c = tf.complex(a_ele, a_azi)
    if polarization_model==1:
        return polarization_model_1(c, theta, phi, slant_angle)
    else:
        return polarization_model_2(c, slant_angle)

In [ ]:
from sionna.rt.antenna import *

In [ ]:
theta = np.linspace(0.0, PI, 50)
phi = np.linspace(-PI, PI, 50)
theta_grid, phi_grid = np.meshgrid(theta, phi, indexing='ij')
c_theta, c_phi = bf_pattern(theta_grid, phi_grid)

In [ ]:
theta[20], phi[20]

In [ ]:
theta_grid[20][20], phi_grid[20][20]

In [ ]:
test_ant2 = Antenna(pattern=bf_pattern, polarization="H",
                 polarization_model=2,
                 dtype=tf.complex64
                )

In [ ]:
sionna.rt.antenna.visualize(bf_pattern)

In [ ]:
sionna.rt.antenna.visualize(dipole_pattern)
sionna.rt.antenna.visualize(hw_dipole_pattern)
sionna.rt.antenna.visualize(iso_pattern)
sionna.rt.antenna.visualize(tr38901_pattern)

In [ ]:

theta = np.linspace(0.0, PI, 50)
phi = np.linspace(-PI, PI, 50)
theta_grid, phi_grid = np.meshgrid(theta, phi, indexing='ij')

phi_grid = tf.where(phi_grid<-PI/2, 0.0, phi_grid)
phi_grid = tf.where(phi_grid>PI/2, 0.0, phi_grid)

print(phi_grid)



c_theta, c_phi = bf_pattern(theta_grid, phi_grid)



In [ ]:
no = 0.1 # noise variance

# Init channel layer
channel = ApplyOFDMChannel(add_awgn=True)

In [ ]:
cm = scene.coverage_map(num_samples=10e5,
                            max_depth=5,
                            diffraction=True,
                            cm_center=[0, 0 ,5.15],
                            cm_orientation=[0,0,0],
                            cm_size=[400,400],
                            cm_cell_size=[1,1])

In [ ]:
scene.preview(coverage_map=cm)

In [ ]:
cm.show();